In [2]:
import torch
torchversion = torch.__version__

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 37.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/QSeer/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/QSeer


In [3]:
import joblib
import numpy as np
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, InMemoryDataset
import torch

def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    super(GraphDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + ".pt"]

  def process(self):
    data_list = []
    my_list = joblib.load(self.filename)
    index = 0
    for item in my_list:
      graph = item["graph"]
      gamma = np.array(item["gamma"], dtype=np.float32)
      beta =  np.array(item["beta"], dtype=np.float32)
      layer = np.array(np.shape(gamma)[0], dtype=np.int32)
      n_values = np.max(4)
      layer = np.eye(n_values)[layer-1]
      label = interleave_arrays(gamma, beta)
      label = np.pad(label, (0, 8 - np.shape(label)[0]), mode='constant', constant_values=0)

      source_list = None
      target_list = None
      for edge in graph.edges():
        if source_list is None:
          source_list = np.array(edge[0])
          target_list = np.array(edge[1])
        else:
          source_list = np.append(source_list, edge[0])
          target_list = np.append(target_list, edge[1])

      edge_index = torch.tensor([source_list, target_list], dtype=torch.int64)
      edge_attr = torch.tensor([[i] for i in range(len(graph.edges()))], dtype=torch.float32)
      x = torch.tensor([[i] for i in range(graph.number_of_nodes())], dtype=torch.float32)
      y = torch.tensor(label, dtype=torch.float32).flatten()
      y = torch.unsqueeze(y, 0)
      layer = torch.tensor(layer, dtype=torch.float32).flatten()
      layer = torch.unsqueeze(layer, 0)
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=y, layer=layer)
      data_list.append(data)

    self.save(data_list, self.processed_paths[0])

train_dataset = GraphDataset(root='./data/', file_name='./data/graph_train.pkl')
test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
valid_dataset = GraphDataset(root='./data/', file_name='./data/graph_valid.pkl')

print(train_dataset.num_node_features)
print(train_dataset.num_classes)
print(train_dataset[0].layer)

1
8
tensor([[1., 0., 0., 0.]])


In [4]:
import joblib
import numpy as np
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, InMemoryDataset
import torch

def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphWDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    super(GraphWDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + ".pt"]

  def process(self):
    data_list = []
    my_list = joblib.load(self.filename)
    index = 0
    for item in my_list:
      source_list = item["slist"]
      target_list = item["tlist"]

      gamma = np.array(item["gamma"], dtype=np.float32)
      beta =  np.array(item["beta"], dtype=np.float32)
      layer = np.array(np.shape(gamma)[0], dtype=np.int32)
      n_values = np.max(4)
      layer = np.eye(n_values)[layer-1]
      label = interleave_arrays(gamma, beta)
      label = np.pad(label, (0, 8 - np.shape(label)[0]), mode='constant', constant_values=0)

      edge_index = torch.tensor([source_list, target_list], dtype=torch.int64)
      edge_attr = torch.tensor([[item["wlist"][i]] for i in range(len(item["wlist"]))], dtype=torch.float32)
      num_nodes = torch.max(edge_index).item() + 1
      x = torch.tensor([[i] for i in range(num_nodes)], dtype=torch.float32)
      y = torch.tensor(label, dtype=torch.float32).flatten()
      y = torch.unsqueeze(y, 0)
      layer = torch.tensor(layer, dtype=torch.float32).flatten()
      layer = torch.unsqueeze(layer, 0)
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=y, layer=layer)
      data_list.append(data)

    self.save(data_list, self.processed_paths[0])


train_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_train.pkl')
test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
valid_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_valid.pkl')

print(train_dataset.num_node_features)
print(train_dataset.num_classes)
print(train_dataset[0].layer)

1
8
tensor([[1., 0., 0., 0.]])


In [12]:
import joblib
import numpy as np
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, InMemoryDataset
import torch

def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphGDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, num_layer = 4, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    self.num_layer = num_layer
    super(GraphGDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + "_" + str(self.num_layer) + "_.pt"]

  def process(self):
    data_list = []
    graph_list = joblib.load(self.filename)
    index = 0
    for graph in graph_list:
      source_list = None
      target_list = None
      edge_list = graph.edges()
      for edge in edge_list:
        if source_list is None:
          source_list = np.array([edge[0]])
          target_list = np.array([edge[1]])
        else:
          source_list = np.append(source_list, edge[0])
          target_list = np.append(target_list, edge[1])

      layer = np.array(self.num_layer, dtype=np.int32)
      n_values = np.max(4)
      layer = np.eye(n_values)[layer-1]

      edge_index = torch.tensor([source_list, target_list], dtype=torch.int64)
      edge_attr = torch.tensor([[i] for i in range(len(edge_list))], dtype=torch.float32)

      x = torch.tensor([[i] for i in range(graph.number_of_nodes())], dtype=torch.float32)
      layer = torch.tensor(layer, dtype=torch.float32).flatten()
      layer = torch.unsqueeze(layer, 0)
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=None, layer=layer)
      data_list.append(data)

    self.save(data_list, self.processed_paths[0])

In [5]:
from torch.nn import Linear, Sequential, ReLU#, Dropout, BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.nn import global_mean_pool

class DGNN(torch.nn.Module):
  def __init__(self, n_x, n_y, dim_h):
    super(DGNN, self).__init__()
    hidden_channels_gcn = dim_h * 2
    hidden_channels_gat = hidden_channels_gcn * 2
    hidden_channels_gin = hidden_channels_gat * 2

    # Initialize the first GCNConv layer in the forward method
    self.conv1 = GCNConv(n_x, hidden_channels_gcn)
    self.hidden_channels_gcn = hidden_channels_gcn
    self.conv2 = GCNConv(hidden_channels_gcn, hidden_channels_gcn)
    self.gat_conv1 = GATConv(hidden_channels_gcn, hidden_channels_gat)
    self.gat_conv2 = GATConv(hidden_channels_gat, hidden_channels_gat)

    mlp = torch.nn.Sequential(
        Linear(hidden_channels_gat, hidden_channels_gin),
        ReLU(),
        Linear(hidden_channels_gin, hidden_channels_gin)
    )
    self.gin_conv1 = GINConv(mlp)
    self.out = Linear(hidden_channels_gin + 4, hidden_channels_gin)
    self.out2 = Linear(hidden_channels_gin, n_y)

  def forward(self, x, edge_index, edge_weight, layer, batch):
    x = F.relu(self.conv1(x = x, edge_index = edge_index, edge_weight = edge_weight))
    #print(x)
    x = F.relu(self.conv2(x = x, edge_index = edge_index, edge_weight = edge_weight))
    x = F.relu(self.gat_conv1(x = x, edge_index = edge_index, edge_attr = edge_weight))
    x = F.relu(self.gat_conv2(x = x, edge_index = edge_index, edge_attr = edge_weight))
    x = F.relu(self.gin_conv1(x = x, edge_index = edge_index))
    x = global_mean_pool(x, batch)
    #print(x.shape)
    #print(layer.shape)
    x = torch.cat([x, layer], dim=1)
    x = self.out(x)
    x = self.out2(x)
    return x

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

train_dataset = GraphDataset(root='./data/', file_name='./data/graph_train.pkl')
test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
valid_dataset = GraphDataset(root='./data/', file_name='./data/graph_valid.pkl')

model = DGNN(n_x=train_dataset.num_node_features, n_y=train_dataset.num_classes, dim_h=256).to("cuda")
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

n_epochs = 20
batch_size = 4096
myloader = DataLoader(train_dataset, batch_size=batch_size)
youloader = DataLoader(valid_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)

best_val_loss = float('inf')
save_path = './best_model.pth'

for epoch in range(n_epochs):
  model.train()
  mse_list = []
  for data in myloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mse_list.append(float(loss))
  train_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} training mse {train_mse:.4f}")

  model.eval()
  mse_list = []
  for data in youloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    mse_list.append(float(loss))
  valid_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} valid mse {valid_mse:.4f}")

  if valid_mse < best_val_loss:
    best_val_loss = valid_mse
    torch.save(model.state_dict(), save_path)
    print(f'Saved best model with validation loss: {best_val_loss:.4f}')

  scheduler.step()

model epoch 0 training mse 43119.5338
model epoch 0 valid mse 0.0027
Saved best model with validation loss: 0.0027
model epoch 1 training mse 0.0018
model epoch 1 valid mse 0.0015
Saved best model with validation loss: 0.0015
model epoch 2 training mse 0.0013
model epoch 2 valid mse 0.0011
Saved best model with validation loss: 0.0011
model epoch 3 training mse 0.0012
model epoch 3 valid mse 0.0008
Saved best model with validation loss: 0.0008
model epoch 4 training mse 0.0009
model epoch 4 valid mse 0.0007
Saved best model with validation loss: 0.0007
model epoch 5 training mse 0.0010
model epoch 5 valid mse 0.0005
Saved best model with validation loss: 0.0005
model epoch 6 training mse 0.0009
model epoch 6 valid mse 0.0005
Saved best model with validation loss: 0.0005
model epoch 7 training mse 0.0008
model epoch 7 valid mse 0.0004
Saved best model with validation loss: 0.0004
model epoch 8 training mse 0.0007
model epoch 8 valid mse 0.0004
Saved best model with validation loss: 0.00

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

train_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_train.pkl')
test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
valid_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_valid.pkl')

model = DGNN(n_x=train_dataset.num_node_features, n_y=train_dataset.num_classes, dim_h=256).to("cuda")
#model.load_state_dict(torch.load(save_path, weights_only=True))

loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

n_epochs = 20
batch_size = 1024
myloader = DataLoader(train_dataset, batch_size=batch_size)
youloader = DataLoader(valid_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)

best_val_loss = float('inf')
save_path = './best_weighted_model.pth'

for epoch in range(n_epochs):
  model.train()
  mse_list = []
  for data in myloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mse_list.append(float(loss))
  train_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} training mse {train_mse:.4f}")

  model.eval()
  mse_list = []
  for data in youloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    mse_list.append(float(loss))
  valid_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} valid mse {valid_mse:.4f}")

  if valid_mse < best_val_loss:
    best_val_loss = valid_mse
    torch.save(model.state_dict(), save_path)
    print(f'Saved best model with validation loss: {best_val_loss:.4f}')

  scheduler.step()

model epoch 0 training mse 31255.4957
model epoch 0 valid mse 0.0127
Saved best model with validation loss: 0.0127
model epoch 1 training mse 0.0036
model epoch 1 valid mse 0.0023
Saved best model with validation loss: 0.0023
model epoch 2 training mse 0.0020
model epoch 2 valid mse 0.0018
Saved best model with validation loss: 0.0018
model epoch 3 training mse 0.0015
model epoch 3 valid mse 0.0013
Saved best model with validation loss: 0.0013
model epoch 4 training mse 0.0013
model epoch 4 valid mse 0.0010
Saved best model with validation loss: 0.0010
model epoch 5 training mse 0.0010
model epoch 5 valid mse 0.0008
Saved best model with validation loss: 0.0008
model epoch 6 training mse 0.0009
model epoch 6 valid mse 0.0006
Saved best model with validation loss: 0.0006
model epoch 7 training mse 0.0008
model epoch 7 valid mse 0.0006
Saved best model with validation loss: 0.0006
model epoch 8 training mse 0.0006
model epoch 8 valid mse 0.0004
Saved best model with validation loss: 0.00

In [9]:
import numpy as np
import joblib

save_path = './best_model.pth'
test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
model = DGNN(n_x=test_dataset.num_node_features, n_y=test_dataset.num_classes, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()
testloader = DataLoader(test_dataset, batch_size=4096)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())

test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, "./test_result.pkl")

(81958, 8)


['./test_result.pkl']

In [10]:
import numpy as np
import joblib

save_path = './best_weighted_model.pth'
test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
model = DGNN(n_x=test_dataset.num_node_features, n_y=test_dataset.num_classes, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()
testloader = DataLoader(test_dataset, batch_size=4096)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())

test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, "./test_result_weighted.pkl")

(9509, 8)


['./test_result_weighted.pkl']

In [14]:
import numpy as np
import joblib
from torch_geometric.data import Batch, Dataset

save_path = './best_model.pth'
test_dataset1 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=1)
test_dataset2 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=2)
test_dataset3 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=3)


model = DGNN(n_x=1, n_y=8, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()

testloader = DataLoader(test_dataset1, batch_size=250)

print(len(testloader))

test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_1.pkl")


testloader = DataLoader(test_dataset2, batch_size=250)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_2.pkl")

testloader = DataLoader(test_dataset3, batch_size=250)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_3.pkl")

12
(3000, 8)
(3000, 8)
(3000, 8)


['./unseen_result_3.pkl']

In [1]:
!pip3 install tensorcircuit[tensorflow]

In [ ]:
import tensorcircuit as tc
import tensorflow as tf
import networkx as nx
import numpy as np
import random
import joblib
import torch

def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

def beta_to_qaoa(beta):
  return np.pi * np.array(beta, dtype=np.float32)

def gamma_to_qaoa(gamma):
  return -np.pi * np.array(gamma, dtype=np.float32) / 2

def all_to_qaoa(param, layer):
  pi0 = [np.pi] * layer
  pi1 = [-np.pi/2] * layer
  pi2 = interleave_arrays(pi1, pi0)
  result = pi2 * param
  return result.astype(np.float32)

def beta_to_qaoa(beta):

    return np.pi * np.array(beta, dtype=np.float32)


K = tc.set_backend("tensorflow")

def QAOAansatz(params, g=None, L=4, return_circuit=False):
  n = len(g.nodes)
  c = tc.Circuit(n)
  for i in range(n):
    c.H(i)

  #print(L)
  for j in range(L):
    for e in g.edges:
      c.exp1(e[0], e[1], unitary=tc.gates._zz_matrix, theta=params[2 * j])

    for i in range(n):
        c.rx(i, theta=params[2 * j + 1])

  if return_circuit is True:
    return c

  loss = 0.0
  for e in g.edges:
    loss += c.expectation_ps(z=[e[0], e[1]])

  return K.real(loss)


QAOA_vvag = K.jit(
  tc.backend.vvag(QAOAansatz, argnums=0, vectorized_argnums=0), static_argnums=(1, 3)
)
opt = K.optimizer(tf.keras.optimizers.Adam(1e-3))

graph_test = joblib.load("./data/graph_test.pkl")
result_test = joblib.load("./test_result.pkl")

index = 0
better = 0
best = 0

for graph in graph_test:
  g = graph["graph"]
  gamma = gamma_to_qaoa(graph["gamma"])
  beta =  beta_to_qaoa(graph["beta"])
  layer = np.shape(gamma)[0]
  label = interleave_arrays(gamma, beta)
  label = np.pad(label, (0, 8 - np.shape(label)[0]), mode='constant', constant_values=0)
  label = np.expand_dims(label, axis=0)
  para = result_test[index]
  para = all_to_qaoa(para, 4)
  para = np.expand_dims(para, axis=0)
  p1 = K.implicit_randn(shape=[1, 8], stddev=0.1) # random
  p2 = tf.convert_to_tensor(label) # ground truth
  p3 = tf.convert_to_tensor(para) # predict
  p = tf.concat([p1, p2, p3], 0)

  LL = []
  for num_circuit in range(3):
    #c = QAOAansatz(params=p[num_circuit], g=prob, return_circuit=True)
    loss = QAOAansatz(params=p[num_circuit], g=g, L=layer)
    LL.append(K.numpy(loss))

  if LL[2] < LL[0]:
    better += 1

  if LL[2] < LL[1]:
    best += 1

  index += 1
  if index % 100 == 0:
    print(index)
    print(better)
    print(best)


print(index)
print(better)
print(best)

100
100
67
